In [1]:
# https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
pip install -U langchain openai chromadb langchainhub bs4

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.9 MB/s eta 0:00:00
  Using cached pydantic-2.5.2-py3-none-any.whl.metadata (65 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 16.1 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.1 MB/s eta 0:00:00
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached flatbuffers-23.5.26-py2.py3-no

In [8]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 7.9 MB/s eta 0:00:00
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.2 MB/s eta 0:00:00
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 

In [20]:
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate

In [2]:
#embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [3]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [4]:
# save to disk
#vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma_db")

# load from disk
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_db")

In [5]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?")

## Step 1. Load

In [8]:
len(docs)

1

In [9]:
len(docs[0].page_content)

42824

In [10]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [8]:
print(docs[0].page_content)



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

In [11]:
print(docs[0])

page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n

## Step 2. Split

In [11]:
len(splits)

68

In [13]:
len(splits[0].page_content)

654

In [18]:
splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'title': "LLM Powered Autonomous Agents | Lil'Log",
 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:',
 'language': 'en'}

## Step 4. Retrieve

In [20]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [24]:
retrieved_docs = retriever.get_relevant_documents(
    "What are the approaches to Task Decomposition?"
)

In [25]:
len(retrieved_docs)

6

In [38]:
retrieved_docs[0].page_content

'Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'

In [41]:
retrieved_docs[2].page_content

'Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'

## Step 5. Generate

In [ ]:
# https://smith.langchain.com/hub/rlm/rag-prompt

In [28]:
print(
    prompt.invoke(
        {"context": "filler context", "question": "filler question"}
    ).to_string()
)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [29]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It can be done through prompting techniques like Chain of Thought (CoT) or Tree of Thoughts, as well as task-specific instructions or human inputs. Task decomposition helps agents plan and execute complicated tasks more effectively.

In [30]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

"Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It can be done through prompting the model to think step by step or by providing task-specific instructions. Task decomposition helps in enhancing model performance and understanding the model's thinking process. Thanks for asking!"

In [31]:
from operator import itemgetter

from langchain.schema.runnable import RunnableParallel

rag_chain_from_docs = (
    {
        "context": lambda input: format_docs(input["documents"]),
        "question": itemgetter("question"),
    }
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)
rag_chain_with_source = RunnableParallel(
    {"documents": retriever, "question": RunnablePassthrough()}
) | {
    "documents": lambda input: [doc.metadata for doc in input["documents"]],
    "answer": rag_chain_from_docs,
}

rag_chain_with_source.invoke("What is Task Decomposition")

{'documents': [{'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:',
   'language': 'en',
   'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
   'title': "LLM Powered Autonomous Agents | Lil'Log"},
  {'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be fra

In [32]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

condense_q_system_prompt = """Given a chat history and the latest user question \
which might reference the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
condense_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
condense_q_chain = condense_q_prompt | llm | StrOutputParser()

In [33]:
from langchain.schema.messages import AIMessage, HumanMessage

condense_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "What is meant by large",
    }
)

'What is the definition of "large" in the context of a language model?'

In [34]:
condense_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "How do transformers work",
    }
)

'How do transformer models function?'

In [35]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def condense_question(input: dict):
    if input.get("chat_history"):
        return condense_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(context=condense_question | retriever | format_docs)
    | qa_prompt
    | llm
)

In [36]:
chat_history = []

question = "What is Task Decomposition?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "What are common ways of doing it?"
rag_chain.invoke({"question": second_question, "chat_history": chat_history})

AIMessage(content='Common ways of task decomposition include:\n\n1. Using Chain of Thought (CoT): CoT is a prompting technique where the model is instructed to "think step by step" and decompose complex tasks into smaller and simpler steps. This technique utilizes more test-time computation and helps the model understand and interpret its thinking process.\n\n2. Task-specific instructions: For certain tasks, specific instructions can be provided to guide the task decomposition process. For example, in the context of writing a novel, the instruction "Write a story outline" can help break down the task into subgoals and steps.\n\n3. Human inputs: Task decomposition can also be done with the help of human inputs. Humans can provide guidance, expertise, or domain-specific knowledge to assist in breaking down the task into manageable subtasks or steps.')

In [ ]:
# cleanup
vectorstore.delete_collection()

# 한글 테스트01 - 위키피디아

In [6]:
loader = WebBaseLoader(
    web_paths=("https://ko.wikipedia.org/wiki/SVB_%ED%8C%8C%EC%82%B0_%EC%82%AC%EA%B1%B4",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("mw-body")
#            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [7]:
print(len(docs))

1


In [8]:
print(docs[0].page_content)







목차 토글







SVB 파산 사건



16개 언어




AfrikaansالعربيةБеларускаяΕλληνικάEnglishEspañolفارسیFrançaisעבריתItalianoBahasa MelayuPortuguêsРусскийTürkçe吴语中文

링크 편집











문서토론





한국어

















읽기편집역사 보기







도구





도구
사이드바로 이동
숨기기



		동작
	


읽기편집역사 보기





		일반
	


여기를 가리키는 문서가리키는 글의 최근 바뀜파일 올리기특수 문서 목록고유 링크문서 정보이 문서 인용하기축약된 URL 얻기위키데이터 항목





		인쇄/내보내기
	


책 만들기PDF로 다운로드인쇄용 판

























위키백과, 우리 모두의 백과사전.





실리콘 밸리 뱅크(SVB)는 지난 3월 10일 뱅크런 사태로 인하여 2일만에 파산하였다. 미국 은행 규모 16위인 실리콘밸리뱅크(SVB)가 사실상 파산 상태에 들어가면서 금융권 전반으로 파장이 확산될지 우려되고 있다. 이 은행 파산은 2008년 금융위기 이후 최대 규모 은행이 무너진 것이고, 미국 역사상으로는 두 번째 규모다.


실리콘밸리은행(SVB) 소개[편집]
1983년 10월 출범한 실리콘밸리은행은 스타트업과 벤처캐피털로부터 예금을 받고 대출 및 투자를 해 혁신생태계 형성·발전 지원에 특화한 지역은행이다. 미국에서 16번째로 컸으며, 실리콘 밸리에서는 가장 큰 은행이었다. 실리콘밸리은행의 주요 고객층은 실리콘밸리에 존재하는 기술중심 스타트업 기업들이었다. SVB는 미국 닷컴버블의 후반 때부터 최근까지 수익이 없던 스타트업 기업들에 대출을 해주었다. 이후 코로나 팬데믹을 거치면서 스타트업들이 빠르게 성장하였고, 투자자들은 이러한 스타트업들에 투자를 하기 시작했다. 스타트업들의 성장은 곧 SVB의 성장을 의미했다.
스타트업 및 투자자의 모든 자금을 흡수한 SVB는 지급준비

In [9]:
print(docs[0])

page_content='\n\n\n\n\n\n목차 토글\n\n\n\n\n\n\n\nSVB 파산 사건\n\n\n\n16개 언어\n\n\n\n\nAfrikaansالعربيةБеларускаяΕλληνικάEnglishEspañolفارسیFrançaisעבריתItalianoBahasa MelayuPortuguêsРусскийTürkçe吴语中文\n\n링크 편집\n\n\n\n\n\n\n\n\n\n\n\n문서토론\n\n\n\n\n\n한국어\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n읽기편집역사 보기\n\n\n\n\n\n\n\n도구\n\n\n\n\n\n도구\n사이드바로 이동\n숨기기\n\n\n\n\t\t동작\n\t\n\n\n읽기편집역사 보기\n\n\n\n\n\n\t\t일반\n\t\n\n\n여기를 가리키는 문서가리키는 글의 최근 바뀜파일 올리기특수 문서 목록고유 링크문서 정보이 문서 인용하기축약된 URL 얻기위키데이터 항목\n\n\n\n\n\n\t\t인쇄/내보내기\n\t\n\n\n책 만들기PDF로 다운로드인쇄용 판\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n위키백과, 우리 모두의 백과사전.\n\n\n\n\n\n실리콘 밸리 뱅크(SVB)는 지난 3월 10일 뱅크런 사태로 인하여 2일만에 파산하였다. 미국 은행 규모 16위인 실리콘밸리뱅크(SVB)가 사실상 파산 상태에 들어가면서 금융권 전반으로 파장이 확산될지 우려되고 있다. 이 은행 파산은 2008년 금융위기 이후 최대 규모 은행이 무너진 것이고, 미국 역사상으로는 두 번째 규모다.\n\n\n실리콘밸리은행(SVB) 소개[편집]\n1983년 10월 출범한 실리콘밸리은행은 스타트업과 벤처캐피털로부터 예금을 받고 대출 및 투자를 해 혁신생태계 형성·발전 지원에 특화한 지역은행이다. 미국에서 16번째로 컸으며, 실리콘 밸리에서는 가장 큰 은행이었다. 실리콘밸리은행의 주요 고객층은 실리콘밸리에 존재하는 기술중심 스타트업 기업들이었다. SVB는 미국 닷컴버블의 

In [10]:
# save to disk
#vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma_db_svb")

# load from disk
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_db_svb")

retriever = vectorstore.as_retriever()

In [22]:
# https://smith.langchain.com/hub/rlm/rag-prompt
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
# Question: {question} 
# Context: {context} 
# Answer:
prompt = hub.pull("rlm/rag-prompt")

template = """You are an assistant for question-answering tasks. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Question: {question}
Context: {context}
Answer: """
prompt_custom = ChatPromptTemplate.from_template(template)

In [23]:

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [24]:
query = "실리콘 밸리 은행 파산 사건 요약해 줘"
retrieved_docs = retriever.get_relevant_documents(
    query
)

In [25]:
len(retrieved_docs)

4

In [26]:
print(retrieved_docs[0].page_content)

실리콘 밸리 뱅크(SVB)는 지난 3월 10일 뱅크런 사태로 인하여 2일만에 파산하였다. 미국 은행 규모 16위인 실리콘밸리뱅크(SVB)가 사실상 파산 상태에 들어가면서 금융권 전반으로 파장이 확산될지 우려되고 있다. 이 은행 파산은 2008년 금융위기 이후 최대 규모 은행이 무너진 것이고, 미국 역사상으로는 두 번째 규모다.


실리콘밸리은행(SVB) 소개[편집]
1983년 10월 출범한 실리콘밸리은행은 스타트업과 벤처캐피털로부터 예금을 받고 대출 및 투자를 해 혁신생태계 형성·발전 지원에 특화한 지역은행이다. 미국에서 16번째로 컸으며, 실리콘 밸리에서는 가장 큰 은행이었다. 실리콘밸리은행의 주요 고객층은 실리콘밸리에 존재하는 기술중심 스타트업 기업들이었다. SVB는 미국 닷컴버블의 후반 때부터 최근까지 수익이 없던 스타트업 기업들에 대출을 해주었다. 이후 코로나 팬데믹을 거치면서 스타트업들이 빠르게 성장하였고, 투자자들은 이러한 스타트업들에 투자를 하기 시작했다. 스타트업들의 성장은 곧 SVB의 성장을 의미했다.
스타트업 및 투자자의 모든 자금을 흡수한 SVB는 지급준비율을 넘어선 충분한 자금이 모이게 되었다. 그리고 자금 상황이 좋아진 스타트업들의 대출사업은 줄어들게 되었다. 이러한 상황에서 SVB는 모아둔 자금과 투자금을 통해 수익을 창출하고자 하였고, 마땅한 투자 상품을 찾던 중 비교적 위험성이 낮은 미국 국채에 투자하게 되었다. 이자는 1%정도였고, 미국 국채는 미국이 망하지 않는 한 수익성이 보장되기에 이와 같은 결정을 내리게 되었다.


In [15]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건 요약해 줘")

'실리콘 밸리 뱅크는 3월 10일 뱅크런 사태로 인해 2일만에 파산하였습니다. 이는 2008년 금융위기 이후 최대 규모의 은행 파산 사례이며, 미국 역사상 두 번째로 큰 규모입니다. 실리콘 밸리 뱅크는 스타트업과 벤처캐피털로부터 예금을 받고 대출 및 투자를 해 혁신생태계 형성·발전 지원에 특화한 지역은행이었습니다.'

In [16]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건 자세히 설명해 줘")

'실리콘 밸리 뱅크(SVB)는 3월 10일 뱅크런 사태로 인해 2일만에 파산하였습니다. 이는 2008년 금융위기 이후 최대 규모의 은행 파산 사례이며, 미국 역사상 두 번째로 큰 규모입니다.'

In [17]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건은 언제 발생했어?")

'실리콘 밸리 은행 파산 사건은 지난 3월 10일에 발생했습니다.'

In [18]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건은 몇 년도에 발생했어?")

'2008년에 발생했습니다.'

In [28]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_custom
    | llm
    | StrOutputParser()
)

In [29]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건 요약해 줘")

'실리콘 밸리 뱅크(SVB)는 3월 10일 뱅크런 사태로 인해 2일 만에 파산하였습니다. 이는 2008년 금융위기 이후 최대 규모의 은행 파산 사례이며, 미국 역사상 두 번째로 큰 규모입니다. SVB는 실리콘 밸리에서 가장 큰 은행으로, 스타트업과 벤처캐피털로부터 예금을 받고 대출 및 투자를 해 혁신생태계 형성·발전을 지원하는 지역은행입니다. SVB는 스타트업들에 대출을 해주었으며, 코로나 팬데믹으로 인해 스타트업들이 빠르게 성장하면서 SVB의 성장을 의미했습니다. 그러나 기준금리 상승으로 인해 경제 상황이 악화되고, 투자금 회수와 스타트업들의 자금 부족으로 SVB는 예금액을 되찾을 수 없게 되었고, 채권을 팔아 부족한 예금액을 돌려줄 수밖에 없었습니다.'

In [33]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건 자세히 설명해 줘")

'실리콘 밸리 뱅크(SVB)는 3월 10일 뱅크런 사태로 인해 2일만에 파산한 미국 은행입니다. 이는 2008년 금융위기 이후 최대 규모의 은행 파산 사례이며, 미국 역사상 두 번째로 큰 규모입니다. 이 파산으로 인해 금융권 전반에 파장이 확산될 우려가 있습니다.'

In [31]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건은 언제 발생했어?")

'실리콘 밸리 은행 파산 사건은 3월 10일에 발생했습니다.'

In [32]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건은 몇 년도에 발생했어?")

'실리콘 밸리 은행 파산 사건은 2008년에 발생했습니다.'

# 한글 테스트02 - 나무위키

In [58]:
from langchain.schema import Document

In [59]:
content="""
1. 개요[편집]
2023년 3월 10일 실리콘밸리 은행이 파산한 사건.
2. 배경[편집]
우선 이 사건을 이해하기 위해서는 실리콘밸리 은행(이하 SVB)이 스타트업들의 은행으로 불리는 매우 특수한 구조라는 점에 착안할 필요가 있다.

스타트업들은 기본적으로 투자자의 돈을 받으며 설립 시 3차, 4차 펀딩까지도 가며 미국 자본시장의 투자력은 어마무시하다. 즉 이들 스타트업은 현금을 많이 들고 있었고 스타트업이 주요 고객인 SVB에는 그 현금들이 그대로 쌓였다. 그런데 스타트업 기업은 일반 기업들과 큰 차이점이 있으니 바로 빚을 내서 인프라에 투자하지 않는다는 점이다. 애초에 어지간하게 큰 대기업이 아닌 이상 펀딩에 성공했다면 당분간 은행에 별도로 빚을 낼 일도 없다. 설령 빚을 내 투자했더라도 스타트업의 특성상 캐시카우를 처음부터 확실히 쥐고 있는 경우가 드물다. 즉 이들 스타트업은 안정 궤도에 오르기까지 실제 벌어들이는 이익이 크지 않다는 점도 특징이다.

이런 특성들은 SVB의 입장, 즉 은행 입장에서는 악영향이다. 은행은 돈을 빌려주고 이자를 받아야 하는데 전통적인 기업은 이것이 가능하다. 은행에서 빚을 내 과감하게 선행 투자를 하고 그로 인해 기업의 이익이 증대되면 이게 곧 은행의 이자 수익으로 이어지는 선순환이다. 그러나 SVB의 주요 고객 포트폴리오는 죄다 현금 부자인 스타트업이었으니 역설적으로 스타트업들의 수익률이 오를수록 SVB에는 현금만 쌓여가는데 이 현금을 빌려줄 곳이 없었다.

사실 여기서 포트폴리오 다변화에 성공하여 다른 산업에도 돈을 빌려주었다면 얘기가 달라졌을 것이다. SVB의 전문가들도 바보는 아니었기 때문에 포트폴리오 다변화를 시도했다. 쌓여있는 현금을 어떻게든 굴려야 은행의 이익으로 이어지는 것은 당연하다. 그런데 SVB가 가장 많이 투자한 분야 중 하나가 미국 국채, 그것도 장기채였다. 저금리가 계속 이어진다는 가정 하에서는 나쁘지 않은 판단이었을 수도 있다. 초우량 안전자산으로 분류되는 미국 국채를 다량 보유하고 있으면 수입이 크지 않지만 안정적으로 계속해서 수익을 올릴 수 있었을 것이다. 그러나 후술하듯이 은행이 자산의 큰 비중을 장기채로 가져가는 것은 드물다 못해 거의 없는 일이며 많은 기사에서 이것을 경영 파탄의 원인으로 지적했다.

그러다가 미국의 인플레이션과 그로 인한 금리 인상이 SVB에게는 악몽으로 다가오고 말았다.

기준금리가 오르면 미국 국채 금리도 같이 인상된다. 채권자들[1]에게는 사실 파티 타임인데 이자율이 오르면 그만큼 자신들의 이자 수익도 늘어난다는 뜻이기 때문이다. 국채 가격의 하락 문제가 있지만 만기까지 보유할 것으로 분류하면 회계상 손실도 인식되지 않는다. 그러나 SVB와 같이 상술한 것처럼 이자율이 낮았던 시절에 미국 국채 장기채를 너무 많이 사들인 상태면 비극이 된다. 이런 상태에서 이자율이 오르니까 이들 장기채는 문자 그대로 똥값이 되고 말았다.[2] 이러면 단기채는 그때그때 높은 수익이 나지만 장기채는 낮은 이율 그대로이므로 수익이 좋지 못하게 된다. 10년 후에 2% 주는 채권이 있고(장기채), 1년 후에 5% 주는 채권(단기채)이 있을 때 그 중에서 고르라면 투자자들은 당연히 후자를 고를 수밖에 없다. 즉 같은 미국 국채라도 현재와 같이 높은 금리 하에서 단기채는 인기가 있어 값이 하락해도 그 폭이 상대적으로 작지만 장기채는 사려는 사람이 없으므로 그 폭이 훨씬 더 크다.

그리고 이 폭락 사태는 그대로 SVB가 보유한 자산 가치 하락으로 이어졌는데 은행의 투자 포트폴리오에서 절반 이상이 장기채인 상황에서[3] 장기채 가격이 바닥에 떨어지고 만 것이다. 그런데 위에 써 놓았듯 SVB는 전통적인 대출이자 수입이 현저히 적은 은행이다. 이러다보니 현금은 죄다 장기채에 묶여 있고 그 장기채는 가치가 하락했으며 다른 수익 나올 구멍도 없다.

게다가 코로나바이러스감염증-19 사태는 SVB의 상황을 더욱 악화시켰다. 코로나 사태로 인해 스타트업의 유동성이 좋아졌다. 코로나로 인한 특수 상황에서 여러 스타트업들이 돈을 많이 벌었고 그 현금은 전부 SVB에 쌓였다. 그러나 코로나가 서서히 종식되면서 스타트업의 코로나 특수 경기는 저물기 시작했고 흘러넘치던 유동성도 점점 제자리로 돌아왔다. 즉 유동성 경색이 시작되었다. 이런 상황에서 위에서 언급한 미국 국채 금리의 상승이 다른 은행과 달리 역설적으로 SVB에는 더 큰 악재로 다가온 것이다.
3. 파산[편집]
2023년 3월 9일 이자율 상승으로 인한 국채 매각으로 18억 달러에 달하는 손해를 입고 주가가 크게 하락했다. 뱅크런이 시작되어 주요 투자자들과 기업이 돈을 빼기 시작했고 3월 10일 주가가 66%가 빠지면서 주식거래가 중지됐다.

SVB가 손실을 공개하고 유상증자 계획을 발표하자마자 투자자들은 SVB의 주식을 팔아치우기 시작했으며 스타트업들도 유동성이 경색되고 안 좋은 소문이 돌자 쌓여 있던 현금을 인출하기 시작했다. 슬랙과 같은 업무용 메신저와 폰뱅킹의 보급화로 뱅크런 속도가 미친 듯이 가속화되어 단 하루만에 56조 원이 인출되었다.# 주가 폭락과 기술력 발전의 시너지 효과로 SVB는 뱅크런이 일어나 자산이 급감하고 결과적으로 SVB의 시가총액이 하루만에 60%가 날아가고 말았다.

이 뱅크런 사태가 매우 빠르게 진행된 원인은 버튼 하나로 예금을 순식간에 옮길 수 있는 스마트폰을 이용한 스마트뱅킹의 점유율이 매우 높은 탓에 뱅크런 속도가 대응할 수 없을 만큼 지나치게 빨라졌기 때문이다. 2008년 세계금융위기보다는 규모가 비교적 작았지만 SNS발 가짜 뉴스 확산에 겁먹은 고객들이 일제히 스마트폰 뱅킹앱을 열고 서둘러 예금을 대량으로 인출하였던 것이 작은 불씨를 더욱 키우고 말았다.

결국 2023년 3월 10일 늦은 밤 캘리포니아 주정부에서 SVB의 은행업허가를 취소[4]하고 파산관재인(receiver)으로 연방예금보험공사(FDIC)를 지정하면서 미국 은행 역사상 두 번째로 큰 규모의 파산 사태가 터졌다.
4. 파산 이후[편집]
연방예금보험공사(FDIC)는 예금자 보호 절차에 들어가 10일 SVB 전 지점을 폐쇄하고 2090억 달러[5]에 달하는 자산을 압류했다.# 실리콘밸리의 경제와 미국 경제 및 세계 경제에 어떤 영향이 갈지 전 세계 언론이 관심을 두었다.#

특히 실리콘밸리에 기반을 둔 스타트업 회사들이 줄도산할 것이라는 우려가 번졌다.#,# 관련 사태로 인해 글로벌 주가와 비트코인 시세도 하락했다.# 이참에 부동산 관련 은행과 부실은행을 정리한다는 말이 돌았지만 2008년 서브프라임 모기지 사태와 다르게 시스템의 위기 자체로 번지지 않을 것이라는 전망도 제기되었다.#

한편 일론 머스크는 본인이 인수할 의사가 있다고 발표했다. #

파산 소식이 퍼지며 전 세계 증시가 3월 13일 월요일 폭락할 것이라는 블랙 먼데이 우려가 번졌다. SVB에 돈이 묶인 한국 기업들과 투자가들도 당황했으며# 국민연금공단도 투자비용으로 보면 큰 부분은 아니지만 10만 주를 가지고 있었다.

한편 뉴욕의 시그니처 은행 역시 실리콘밸리 은행이 파산한 지 며칠도 채 안 된 한국시간으로 3월 13일 오전 파산선고가 내려졌다. # 소비자들 입장에서 연쇄적인 도미노 파산을 심히 우려하게 되었다. 퍼스트리퍼블릭 은행, 팩웨스트 뱅코프, 웨스턴 얼라이언스 뱅코프 등 지역 은행의 주가들도 폭락하면서 위기에 놓였다.

미국 정부는 실리콘밸리은행에 고객이 예치한 돈을 보험 대상 한도와 상관 없이 전액 보증하는 대책을 마련하였다. 재무부, 연방준비제도, 연방예금보험공사가 발표한 공동 성명에 따르면 모든 예금주는 현지 시각 13일부터 예금 전액에 접근할 수 있다.[6][7] 한편 예금에 대해서는 보장한다고 했지만 주식과 채권은 보장하지 않는다고 한다. 그래서 한국 국민연금도 돈을 물린 것으로 전해졌다.#

3월 13일, HSBC가 SVB 영국 법인을 상징적인 금액인 1파운드에 인수하기로 했다. 이에 따라 영국의 거래 기업들의 유동성 문제가 해소될 가능성이 생겼다. #

SVB 파산 사태에도 불구하고 연방준비제도 (FOMC)가 금리를 0.25% 올려 베이비 스탭을 강행했다. 높은 금리 때문에 자금 조달이 어려워진 것이 SVB 사태의 원인으로 꼽혀서 연준이 금리를 올릴지 말지 귀추가 주목되었는데 금리 인상은 결국 멈추지 않았다. 다만 제롬 파월 의장은 이번 금리 인상이 마지막일 수 있다는 의미심장한 메시지를 전달했다. 원래 5.5 ~ 6%까지 금리를 올리려고 했으나 SVB 사태로 인해 5% 선에서 장기간 동결하는 식으로 FOMC가 보다 온건적인 정책으로 선회한 것이 아니냐는 분석도 제기되었다.

3월 27일, 노스캐롤라이나에 본사가 있는 퍼스트 시티즌 은행이 실리콘밸리 은행을 인수하기로 결의하고 실행하였다.#

5월 1일, 퍼스트리퍼블릭 은행이 결국 파산하였다는 소식이 전해졌다. 동시에 JP모건 체이스가 퍼스트리퍼블릭 은행을 인수하겠다고 발표했다.

10월 3일, 실리콘밸리 파이낸셜 그룹의 투자은행 부문이 Jeff Leerink 주도하의 컨소시엄에 매각하는 절차가 완료되었다.
5. 향후 전망[편집]
5.1. 낙관적 전망[편집]
금융업계 및 정부 경제 당국에서는 SVB의 파산이 서브프라임 모기지 사태와 다르게 금융업 전체로의 위기로 번지지는 않을 거라는 전망이 우세했다. 은행의 파산은 확실한 악재이기 때문에 단기적으로 금융시장에 충격을 줄 수 있고 그것이 전체적인 주가 하락과 암호화폐 하락 등으로 이어지기는 했지만 SVB의 포트폴리오가 워낙 특수했기 때문에 다른 은행들은 SVB처럼 망하지는 않을 거라는 게 일반적인 금융 전문가들의 시각이다. 애초에 금리가 오르면서 다른 은행들은 이자 파티를 벌이게 되었으므로 어지간하게 어디 묶여 있지 않은 이상은 망할 가능성은 적다.[8]
5.2. 비관적 전망[편집]
SVB의 파산 사태가 금융권 전체로 충격이 퍼질 것이란 시각도 있는데 이는 전통적인 위기관리 시스템인 신용등급 평가가 이 사례에서 무력화되었음이 드러났기 때문이다. 일주일 전까지 신용등급이 A3였던 은행이 일주일도 안 돼서 C로 폭락하면서 파산한 것이 이 사건이다. 즉 신용등급으로도 부실 기업이나 은행을 거를 수 없다면 무엇을 신뢰하고 투자할 수 있겠는가라는 의문이 제기된 것. 이것 때문에 JP 모건조차도 믿을 수 없다거나 망할 수 있다는 얘기가 나오기도 했다. 이는 2008년 대침체와 월가 점령 시위 이후에도 미국 금융계가 사회 구성원들의 신뢰를 완전히 회복하지 못했음을 나타낸다.

실제로 SVB 파산 고작 3일 만인 3월 13일 시그니처 은행이 추가로 파산선고를 받으면서 심상치 않은 흐름이 보였다. # 다만 시그니처 은행은 전통적인 은행과 달리 암호화폐 전문 은행이었던 만큼 그 여파가 전통적인 은행이 무너지는 것만큼 크지 않으리라는 예측도 있었다. 일단 연방예금보험공사는 SVB와 마찬가지로 시그니처 은행의 자산을 넘겨받아 예금 지급에 최선을 다하겠다고 밝혔지만 얼마 지나지 않아 퍼스트리퍼블릭 은행 역시 파산 위기에 몰리면서 사태가 확대될 조짐이 나왔는데 5월 1일 퍼스트리퍼블릭 은행이 최종 파산하면서 실제로 사태가 확대되었다.

연준이 긴급자금지원을 했지만 만약 퍼스트 시티즌 뱅크에게 인수된 실리콘밸리 은행과 JP모건 체이스에게 인수된 퍼스트리퍼블릭 은행처럼 찰스 슈왑# 마저 파산한다면 여파가 여타 대형 은행까지 더 크게 번질 가능성도 생겼다.
6. 기타[편집]
은행이 자산 배치와 사업 구조를 다변화(diversify)하지 않을 경우 아무리 보수적으로 기업 구조를 짜더라도 은행 자체가 망할 수 있음을 증명한 희귀한 사례가 되었다. 금융기관이 몰락하는 것은 지나치게 공격적인(위험한) 투자 일변도로 포트폴리오를 구성해서 일어나는 경우가 많은데 이처럼 그 반대의 경우도 가능함을 보여준 셈이다. 투자의 제1계명인 계란을 한 바구니에 담지 말라는 대원칙을 위반했기 때문에 일어난 일이다.
이 사건은 뱅크런이 발생할 경우 투자은행이 주도하여 채권 투자와 수신 영업을 통해 원금과 이자 지급을 가능케 하는 발행어음형 CMA의 안정성을 담보할 수 없다는 점을 시사하며 예금자 보호가 되는 은행 예금 및 종합금융형 CMA와 예금자 보호가 되지 않지만 AA 등급 이상의 채권을 담보로 하는 RP형 CMA도 원금 손실 가능성은 없지만 금융기관이 파산할 경우 돈이 묶이는 성가신 상황이 발생할 수 있다는 사실을 상기시켰다. 오히려 예금자 보호가 되지는 않더라도 금융기관의 파산 여부가 유동성에 영향을 끼치지 않는 MMW형 CMA가 오히려 은행 예금이나 다른 CMA 상품보다 더 안전할 수 있다. MMW형 CMA는 운용 주체가 중권 금융기관인 한국증권금융이고 예금자보호법 적용은 받지 않지만 자본시장법 74조에 근거하여 원금과 이자뿐만 아니라 상시 유동성 그 자체를 보장하도록 운용하고 있다. 즉, 불특정 고객이 자기가 맡긴 돈을 아무 때나 출금하더라도 문제가 발생하지 않도록 예탁금을 운용해야 한다. 한국증권금융이 이런 책임을 져버릴 경우 무조건 국가가 배상해야 한다. 은행 예금은 예금자 보호법에 의해 원금과 이자를 합해서 5천만원까지 보장이 되지만 실시간 지급이 보장된다는 뜻은 아니다. 즉, MMW형 CMA가 은행 예금보다 뱅크런 위험에서는 더 안전함을 시사하는 셈이다.[9]
여배우 샤론 스톤이 파산 사태로 절반이나 되는 자산을 날렸다고 한다.
미국 실리콘밸리은행(SVB)의 뱅크런은 소셜미디어 트위터가 촉발했다는 연구 논문이 나왔다.
"""
print(len(content))
doc_init = Document(page_content=content, metadata={'source': 'https://namu.wiki/w/%EC%8B%A4%EB%A6%AC%EC%BD%98%EB%B0%B8%EB%A6%AC%20%EC%9D%80%ED%96%89%20%ED%8C%8C%EC%82%B0', 'title': '실리콘밸리 은행 파산 - 나무위키', 'language': 'No language found.'})

6539


In [60]:
docs = [doc_init]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [61]:
print(len(splits))

1


In [62]:
print(splits[0])

page_content='1. 개요[편집]\n2023년 3월 10일 실리콘밸리 은행이 파산한 사건.\n2. 배경[편집]\n우선 이 사건을 이해하기 위해서는 실리콘밸리 은행(이하 SVB)이 스타트업들의 은행으로 불리는 매우 특수한 구조라는 점에 착안할 필요가 있다.\n\n스타트업들은 기본적으로 투자자의 돈을 받으며 설립 시 3차, 4차 펀딩까지도 가며 미국 자본시장의 투자력은 어마무시하다. 즉 이들 스타트업은 현금을 많이 들고 있었고 스타트업이 주요 고객인 SVB에는 그 현금들이 그대로 쌓였다. 그런데 스타트업 기업은 일반 기업들과 큰 차이점이 있으니 바로 빚을 내서 인프라에 투자하지 않는다는 점이다. 애초에 어지간하게 큰 대기업이 아닌 이상 펀딩에 성공했다면 당분간 은행에 별도로 빚을 낼 일도 없다. 설령 빚을 내 투자했더라도 스타트업의 특성상 캐시카우를 처음부터 확실히 쥐고 있는 경우가 드물다. 즉 이들 스타트업은 안정 궤도에 오르기까지 실제 벌어들이는 이익이 크지 않다는 점도 특징이다.\n\n이런 특성들은 SVB의 입장, 즉 은행 입장에서는 악영향이다. 은행은 돈을 빌려주고 이자를 받아야 하는데 전통적인 기업은 이것이 가능하다. 은행에서 빚을 내 과감하게 선행 투자를 하고 그로 인해 기업의 이익이 증대되면 이게 곧 은행의 이자 수익으로 이어지는 선순환이다. 그러나 SVB의 주요 고객 포트폴리오는 죄다 현금 부자인 스타트업이었으니 역설적으로 스타트업들의 수익률이 오를수록 SVB에는 현금만 쌓여가는데 이 현금을 빌려줄 곳이 없었다.\n\n사실 여기서 포트폴리오 다변화에 성공하여 다른 산업에도 돈을 빌려주었다면 얘기가 달라졌을 것이다. SVB의 전문가들도 바보는 아니었기 때문에 포트폴리오 다변화를 시도했다. 쌓여있는 현금을 어떻게든 굴려야 은행의 이익으로 이어지는 것은 당연하다. 그런데 SVB가 가장 많이 투자한 분야 중 하나가 미국 국채, 그것도 장기채였다. 저금리가 계속 이어진다는 가정 하에서는 나쁘지 않은 판단이었을 수도 있다. 초우량 안전자산으로 분류되는

In [64]:
# save to disk
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma_db_svb05")

# load from disk
#vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_db_svb04")

In [67]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 1})

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [68]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건 요약해 줘")

'2023년 3월 10일, 실리콘밸리 은행이 파산하였고 이는 이자율 상승으로 인한 국채 매각으로 인한 손해와 스타트업들의 유동성 경색 등이 원인이었다. 파산 이후에는 연방예금보험공사(FDIC)가 예금자 보호 절차에 들어가 은행을 폐쇄하고 자산을 압류하였다. 이 사태로 인해 실리콘밸리의 경제와 미국 경제에도 영향을 미쳤다.'

In [69]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건 자세히 설명해 줘")

'2023년 3월 10일, 실리콘밸리 은행이 파산한 사건입니다. 이 사건은 은행의 포트폴리오 구조와 자산 배치의 문제로 인해 발생하였습니다. 파산 이후에는 다른 은행들에도 영향을 미칠 수 있는 위기 상황이었습니다.'

In [47]:
rag_chain.invoke("실리콘 밸리 은행 파산 사건은 언제 발생했어?")

'2023년 3월 10일에 실리콘밸리 은행이 파산했습니다.'